In [10]:
#Version del scraper de Maldita 2.0
#Esta versión ha sido desarrollada para de una forma limpia poder scrapear maldita seleccionando un página en concreto.

In [11]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle
import unicodedata

In [12]:
pagina = 4

In [13]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [14]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(unicodedata.normalize("NFKD", e.get_text()))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Maldita.es"}
    
    return article

In [15]:
lista_enlaces = []
suficiente = True

#Cargo las URLS.
print('Cargando URLs...')

link =  "https://maldita.es/malditobulo/page/" + str(pagina) + "/"

arrayurls= get_urls(link, lista_enlaces)
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/06/19/javier-echevarria-opus-discapacitados-inferiores/', 'https://maldita.es/malditobulo/2020/06/19/amlo-secuestros-evitan-sociedad-pobre/', 'https://maldita.es/malditobulo/2020/06/19/vacuna-coronavirus-covid-19-perros/', 'https://maldita.es/malditobulo/2020/06/19/video-torre-eiffel-espectaculo-luces-homenaje-victimas-covid-19/', 'https://maldita.es/malditobulo/2020/06/18/bulos-y-datos-sobre-las-ayudas-al-alquiler-en-la-comunidad-de-madrid-a-marroquies-y-magrebies/', 'https://maldita.es/malditobulo/2020/06/18/cartel-campana-ayuntamiento-valencia-orgullo-lgtb/', 'https://maldita.es/malditobulo/que-sabemos-acerca-de-la-imagen-que-alerta-sobre-los-componentes-de-las-vacunas-y-explica-por-que-ninguno-de-ellos-deberia-entrar-en-tu-cuerpo', 'https://maldita.es/malditobulo/2020/06/18/correo-facebook-cuenta-bloqueada-malware/', 'https://maldita.es/malditobulo/2020/06/18/bulos-bill-gates-coronavirus/', 'https://maldita.es/malditobulo/20

In [16]:
lista_articulos

[{'Title': '¿Qué sabemos de la supuesta frase del prelado del Opus Dei Javier Echevarría sobre las personas discapacitadas y "los padres que no han mantenido la pureza antes del matrimonio"?',
  'Publication_date': '19/06/2020',
  'Content': 'Desde hace años circula una imagen del que fuera prelado del Opus Dei Javier Echevarría, a quien atribuyen la siguiente frase: "Los discapacitados y subnormales son seres inferiores como castigo de Dios a sus padres pecadores". Aunque no hay pruebas de que pronunciara esas palabras, algunos medios italianos sí se hicieron eco en 1997 de la segunda parte de la frase: "El 90% de los discapacitados son hijos de padres que no han mantenido la pureza antes del matrimonio". Echevarría negó posteriormente en una entrevista haber realizado esa afirmación ya que no respondía a su "pensamiento como hombre, como cristiano y como sacerdote".En un artículo publicado el 11 de abril de 1997, el periódico italiano Corriere della Sera recogió las reaccio

In [17]:
#Guardamos los datos:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Maldita_FN_Pag-' + str(pagina) + '_'+ date
print(filename)

outfile = open(filename,'wb')
pickle.dump(lista_articulos, outfile)  
    
outfile.close()

Maldita_FN_Pag-4_01-07-2020


In [18]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': '¿Qué sabemos de la supuesta frase del prelado del Opus Dei Javier Echevarría sobre las personas discapacitadas y "los padres que no han mantenido la pureza antes del matrimonio"?',
  'Publication_date': '19/06/2020',
  'Content': 'Desde hace años circula una imagen del que fuera prelado del Opus Dei Javier Echevarría, a quien atribuyen la siguiente frase: "Los discapacitados y subnormales son seres inferiores como castigo de Dios a sus padres pecadores". Aunque no hay pruebas de que pronunciara esas palabras, algunos medios italianos sí se hicieron eco en 1997 de la segunda parte de la frase: "El 90% de los discapacitados son hijos de padres que no han mantenido la pureza antes del matrimonio". Echevarría negó posteriormente en una entrevista haber realizado esa afirmación ya que no respondía a su "pensamiento como hombre, como cristiano y como sacerdote".En un artículo publicado el 11 de abril de 1997, el periódico italiano Corriere della Sera recogió las reaccio